<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introducción" data-toc-modified-id="Introducción-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introducción</a></span></li><li><span><a href="#Regularización" data-toc-modified-id="Regularización-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Regularización</a></span><ul class="toc-item"><li><span><a href="#En-el-capítulo-anterior..." data-toc-modified-id="En-el-capítulo-anterior...-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>En el capítulo anterior...</a></span></li><li><span><a href="#Regresión-Ridge" data-toc-modified-id="Regresión-Ridge-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Regresión <em>Ridge</em></a></span><ul class="toc-item"><li><span><a href="#Valores-de-los-parámetros" data-toc-modified-id="Valores-de-los-parámetros-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Valores de los parámetros</a></span></li><li><span><a href="#Función-de-error-modificada" data-toc-modified-id="Función-de-error-modificada-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Función de error modificada</a></span></li><li><span><a href="#Shrinkage" data-toc-modified-id="Shrinkage-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Shrinkage</a></span></li></ul></li><li><span><a href="#Lasso" data-toc-modified-id="Lasso-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Lasso</a></span></li><li><span><a href="#Interpretación-geométrica" data-toc-modified-id="Interpretación-geométrica-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Interpretación geométrica</a></span></li></ul></li><li><span><a href="#Bias-Variance-tradeoff" data-toc-modified-id="Bias-Variance-tradeoff-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bias-Variance tradeoff</a></span><ul class="toc-item"><li><span><a href="#Cálculo-completo-de-sesgo-y-varianza" data-toc-modified-id="Cálculo-completo-de-sesgo-y-varianza-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Cálculo completo de sesgo y varianza</a></span></li></ul></li></ul></div>

# Introducción

Vimos cómo el uso de un modelo demasiado complejo para describir un conjunto de datos de tamaño limitado puede provocar un sobreajuste. Hasta ahora, hemos considerado el *overfitting* como una situación en la que el rendimiento del modelo en el conjunto de entrenamiento es mucho mejor que en los conjuntos de test o validación. Pero hay mucho más.

También discutimos cómo al elegir cuidadosamente los hiperparámetros del modelo, se puede alcanzar un nivel adecuado de complejidad del modelo.

Sin embargo, en muchas situaciones, nos gustaría mantener un modelo complejo para capturar las sutilezas del proceso de generación de datos. En el ejemplo de la semana pasada, que reconsideraremos acá con ciertas modificaciones, no hay un grado de polinomio que realmente capture el proceso correcto detrás de los datos. En otras palabras, el proceso real no está incluído en el subespacio generado por las funciones de base. Idealmente, necesitaríamos un polinomio de alto grado para hacer esto. Pero en ese caso, también debemos evitar que el modelo aprenda de memoria el ruido de la realización de los datos dados e incurra en un sobreajuste.

En este *notebook*, primero describiremos la **regularización**, una técnica para controlar la complejidad de los modelos sin reducir por completo su flexibilidad, y luego analizaremos de manera diferente la idea de sobreajuste y complejidad del modelo, introduciendo el concepto de **sesgo** y **varianza** de un modelo, y discutiendo su **balance** (*tradeoff*) natural. Estas ideas nos acompañarán durante el resto del viaje y serán clave para comprender el funcionamiento interno de algunas técnicas de aprendizaje automático, como los métodos de conjunto.

<font size = 3> **¡Empecemos!** </font>

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import sys

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "05_Regularization"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "plots", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    os.makedirs(IMAGES_PATH, exist_ok=True)
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Regularización

## En el capítulo anterior...

Comenzamos recreando un dataset con propiedades similares al del martes.

In [ ]:
from sklearn.model_selection import train_test_split

N_SAMPLES = 20
x = np.linspace(0,1,num=N_SAMPLES).reshape(-1,1)

np.random.seed(42)

def ground_truth(x):
    return 4 * x + np.sin(x*6)

t = ground_truth(x) + 0.5 * np.random.randn(N_SAMPLES,1)

x_train, x_test, t_train, t_test = train_test_split(x, t, random_state=42)

In [ ]:
plt.scatter(x_train, t_train, c='blue', label='train')
plt.scatter(x_test, t_test, c='red', label='test')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Habíamos visto que ajustar un polinomio de orden superior (por encima de 4, digamos) conduce a una capacidad de generalización muy pobre. Esto se ve al evaluar las capacidades de predicción del modelo en el conjunto de testeo.

In [ ]:
from sklearn.metrics import mean_squared_error

# Define practical equations.
def plot_data(model, x, y, x_test=[], y_test=[]):
    # Plot data
    plt.scatter(x, y, c='red', label='train')
    
    # Test is present
    if len(x_test) > 0:
        assert len(x_test) == len(y_test), "Test dataset size incompatible"
        plt.scatter(x_test, y_test, c='blue', label='test')

    # Define oversample x array for plotting
    x_ = np.linspace(0,1, 100).reshape(-1,1)

    # Make model prediction on oversampled array
    predictions = model.predict(x_)
    
    # Plot predicted curve
    plt.plot(x_, predictions, c='green', label='predictions', lw=4, alpha=0.8)

    plt.legend()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()
    return
    
def compute_errors(model, x_train, y_train, x_test=None, y_test=None, print_result=True):
    """Compute and print MSE for training and, if given, test datasets."""
    training_error = mean_squared_error(y_train, model.predict(x_train))
    if print_result: print(f"The training mse is: {round(training_error,2)}")
    
    if x_test is not None and y_test is not None:
        test_error = mean_squared_error(y_test, model.predict(x_test))
        if print_result: print(f"The test mse is: {round(test_error,2)}")
        return training_error, test_error
    else:
        return training_error

In [ ]:
# Define function to perform pipeline for a given degree.
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

def polynomial_regressor(M):
    pr = Pipeline([
        ('poly_features', PolynomialFeatures(M)),
        ('regressor', LinearRegression(fit_intercept=False) )])
    return pr

In [ ]:
degrees = range(1, 10)
train_errors = []
test_errors = []
models = []

for M in degrees:
    print(f"Polynomial degree: {M}")
    pr = polynomial_regressor(M)
    pr.fit(x_train, t_train)
    
    train_e, test_e = compute_errors(pr, x_train, t_train, x_test, t_test, print_result=True)
    
#     plot_data(pr,x_train,y_train, x_test, y_test)
    
    train_errors.append(train_e)
    test_errors.append(test_e)
    models.append(pr)

In [ ]:
# Perform multi-plot
ncolumns = 3

fig = plt.figure(figsize=(14, 14))

if len(models) % ncolumns == 0:
    extrarow = 0
else:
    extrarow = 1
        
axs = fig.subplots(ncols=ncolumns, nrows=int(np.floor(len(models)/ncolumns) + extrarow))

x_ = np.linspace(0,1, 100).reshape(-1,1)
for i, ax in zip(range(len(models)), axs.flatten()):
    ax.plot(x_train, t_train, 'o', ms=10, mfc='None', label='Train')
    ax.plot(x_test, t_test, 'or', ms=10, mfc='None', label='Test')
    ax.plot(x_, models[i].predict(x_), 'r-', lw=3, alpha=0.8, label='Predicted curve')
    ax.plot(x_, ground_truth(x_), 'k-', lw=3, alpha=0.5, label='Ground truth')
    #
    ax.set_title('Degree: {}'.format(models[i]['poly_features'].degree))
    #
    ax.set_ylim(0, 3.9)
    
# Make a single legend
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=len(handles), 
           fontsize=mpl.rc_params()['axes.labelsize'], borderaxespad=2.5)

Hemos visto que el polinomio con el error de testeo más bajo fue el de grado cuatro. Al inspeccionar la curva, parece ser la que mejor se aproxima al proceso real.

In [ ]:
plt.figure(figsize=(9, 6))
plt.semilogy(range(1, len(test_errors)+1), test_errors, '-or', mfc='None', ms=10, label='Test Error')
plt.semilogy(range(1, len(train_errors)+1), train_errors, '-ob', mfc='None', ms=10, label='Train Error')
plt.legend(loc='upper left')

## Regresión *Ridge*

El sobreajuste es un subproducto de la búsqueda (*obsesiva*, se podría decir) de minimizar una función de error utilizando modelos muy flexibles. Parece claro que podemos ganar algo imponiendo algunos límites a la función de error.

La escribimos:

$$
MSE(\boldsymbol{\omega}) = \frac{1}{2} \sum_{i=1}^{N} \left\{y(x_i, \boldsymbol{\omega}) - t_i\right\}^2\;\;.
$$

**¿Cómo deberíamos modificar esta expresión?** 

Podemos obtener algunas pistas estudiando los valores de los parámetros de la regresión polinomial.

### Valores de los parámetros

In [ ]:
coef = np.full((len(models)+1, len(models)), np.nan)
for i in range(len(models)):
    coef[:i+2, i] = models[i]['regressor'].coef_[0]

coef_df = pd.DataFrame(coef, columns=range(1, len(models)+1))
coef_df

***
**Pregunta**

* ¿Qué ven?

### Función de error modificada

En vistas de que los valores de los coeficientes aumentan dramáticamente cuando empezamos a sobreajustar, podemos pensar en incluir una penalización para valores muy grandes de los parámetros. Escribimos, entonces, una nueva función de error:

$$
E_\text{ridge}(\boldsymbol{\omega}; \lambda) = \frac{1}{2} \sum_{i=1}^{N} \left\{y(x_i, \boldsymbol{\omega}) - t_i\right\}^2
 + \frac{\lambda}{2} \sum_{i=1}^M \omega_i^2\;\;, 
$$
donde $\lambda$ se llama el término de regularización (o de penalización). El segundo término es equivalente al cuadrado de la norma $L2$ del vector de parámetros. Es decir,

$$
||\boldsymbol{\omega}||^2 = \boldsymbol{\omega}^T \boldsymbol{\omega} = \left(\omega_1 \ldots \omega_M\right)\begin{pmatrix}\omega_1 \\ \vdots \\ \omega_M\end{pmatrix} = \sum_{i=1}^M \omega_i^2\;\;.
$$

El parámetro de regularización constituye un nuevo hiperparámetro del modelo (**¿además de qué?**) y debe determinarse, por ejemplo, mediante validación cruzada.

Con este término adicional, vemos que los valores de parámetros muy grandes serán penalizados. Una de las mejores cosas de esta regresión, llamada **regresión de Ridge**, es que todavía existe una forma analítica de encontrar los parámetros que minimizan la función de error modificada. Esta es una consecuencia directa del uso de la norma $L2$, que es solo la suma de cuadrados de los parámetros del modelo. 

Por otro lado, noten que estos valores ya no constituyen los estimadores de máxima verosimilitud del problema, como vimos la otra vez. Perderemos algunas de las propiedades agradables que tienen los estimadores de máxima verosimilitud (por ejemplo, el hecho de ser insesgados). Pero ganaremos mucho. En particular, una varianza reducida.

Veamos cómo funciona esto en la práctica. Ajustemos nuevamente un polinomio de noveno grado a los datos, pero esta vez usando un regresor regularizado.

In [ ]:
from sklearn.linear_model import Ridge

def ridge(m, lam):
    return Pipeline([('poly_features', PolynomialFeatures(degree=m)),
                     ('regressor', Ridge(alpha=lam/2.0, fit_intercept=False))])

# Fit Ridge
rr = ridge(degrees[-1], 0.001)
rr.fit(x_train, t_train)

# Fit OLS
ll = polynomial_regressor(degrees[-1])
ll.fit(x_train, t_train)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.plot(x_train, t_train, 'o', ms=10, mfc='None', label='Train')
ax.plot(x_, rr.predict(x_), 'r-', lw=3, alpha=0.8, label='Predicted curve (ridge)', zorder=2)
ax.plot(x_, ll.predict(x_), 'g-', lw=1, alpha=0.8, label='Predicted curve (OLS)', zorder=0)
ax.plot(x_, ground_truth(x_), 'k-', lw=3, alpha=0.5, label='Ground truth', zorder=1)
    #
ax.set_title('Degree: {}; $\lambda$: {:.2e}'.format(rr['poly_features'].degree, 
                                                    rr['regressor'].alpha *2), fontsize=16)
    #
ax.set_ylim(0, 3.9)
ax.legend(loc='best', fontsize=14)

In [ ]:
rr['regressor'].coef_[0]

Gracias al término de regularización, la curva es menos ondulada, y nos da la impresión de que debería producir una mejor generalización.

In [ ]:
print('OLS; degree 9')
print('The training mse is: {:.2f}'.format(train_errors[-1]))
print('The test mse is: {:.2f}'.format(test_errors[-1]))
print('####')
print('Ridge Regression; degree {}, lambda = {}'.format(rr['poly_features'].degree, 
                                                        rr['regressor'].alpha *2))
train_e, test_e = compute_errors(rr, x_train, t_train, x_test, t_test, print_result=True)

### Shrinkage

Estas técnicas a veces se denominan **shrinkage** (contracción o encojimiento), porque hacen que los parámetros del modelo se reduzcan a cero. Como veremos más adelante, esto hace que el modelo cambie algo varianza por algo de sesgo.

Veamos la evolución de los valores de los parámetros a medida que pasamos de valores pequeños a grandes del término de penalización.

In [ ]:
# Grid values of lambda
lls = np.logspace(-5, 2, 100)

cc = []

# Iterate over values, fit, and record coefficient values
for ll in lls:
    rr = ridge(degrees[-1], ll)
    rr.fit(x_train, t_train)
    cc.append(rr['regressor'].coef_[0])

cc = np.array(cc)

In [ ]:
cc.shape

In [ ]:
#Plot coefficient amplitudes versus regularization parameter.
fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(111)

for i in range(len(cc[0])):
    ax.semilogx(lls, cc[:, i], label='$\omega_{{{}}}$'.format(i), lw=3, alpha=0.6)
ax.legend(ncol=3, fontsize=16)
ax.set_xlabel('$\lambda$')
ax.set_ylabel('Parameter value')

## Lasso

Una regresión regularizada diferente que se usa a menudo es la **regresión LASSO**, que naturalmente selecciona las variables más relevantes y produce modelos más parsimoniosos.

En lugar de penalizar la función de error usando la suma de cuadrados de los parámetros del modelo como hicimos más arriba, **LASSO** usa la norma $l1$, que es simplemente la suma de los *valores absolutos* de los parámetros del modelo.

En otras palabras, la norma $l1$ de un vector es simplemente:

$$
||\boldsymbol{\omega}||_1 = \sum_i |\omega_i|\;\;.
$$

Por tanto, la función de error modificada es

$$
E_\text{lasso}(\boldsymbol{\omega}; \lambda) = \frac{1}{2} \sum_{i=1}^{N} \left\{y(x_i, \boldsymbol{\omega}) - t_i\right\}^2
 + \frac{\lambda}{2} \sum_{i=1}^M |\omega_i|\;\;, 
$$

donde hemos vuelto a introducir el hiperparámetro $\lambda$ para controlar el nivel de penalización.

La primera consecuencia de esta elección de penalización es que la función de error ya no puede optimizarse (minimizarse) de manera analítica. Hay que recurrir, entonces, a diferentes algoritmos iterativos.

En `sklearn`, existen al menos dos implementaciones:

* `linear_model.Lasso`, que usa *coordinate descent* para encontrar el mínimo.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e3/Coordinate_descent.svg/500px-Coordinate_descent.svg.png" width=500px></img>

* `linear_model.LassoLars`, que usa LARS (least angle regression), que es similar a forward stepwise regression (es decir, se empieza con los coeficientes en cero, $\boldsymbol{\omega} = 0$, y se van aumentando prograsivamente.

In [ ]:
from sklearn.linear_model import Lasso, LassoLars

def lasso(m, lam):
    return Pipeline([('poly_features', PolynomialFeatures(degree=m)),
                     ('regressor', Lasso(alpha=lam/2.0, fit_intercept=False, 
                                         max_iter=500000))])

rr = lasso(9, 0.001)
rr.fit(x_train, t_train)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.plot(x_train, t_train, 'o', ms=10, mfc='None', label='Train')
ax.plot(x_, rr.predict(x_), 'r-', lw=3, alpha=0.8, label='Predicted curve')
ax.plot(x_, ground_truth(x_), 'k-', lw=3, alpha=0.5, label='Ground truth')
    #
ax.set_title('Degree: {}; $\lambda$: {:.2e}'.format(rr['poly_features'].degree, 
                                                    rr['regressor'].alpha *2), fontsize=16)
    #
ax.set_ylim(0, 3.9)
ax.legend(loc=0, fontsize=15)

At first look, it seems that both regressors produce the same results. But in reality there is a _big_ difference between both methods.

Let's redo the shrinkage plot.

In [ ]:
rr['regressor'].coef_

In [ ]:
# Grid values of lambda
lls = np.logspace(-4, 1, 100)

# Polynomial degree
deg = 9

cc_lasso = np.empty([len(lls), deg+1])

# Iterate over values, fit, and record coefficient values
for i, ll in enumerate(lls):
    #print(ll)
    rr = lasso(deg, ll)
    rr.fit(x_train, t_train)
    cc_lasso [i] =rr['regressor'].coef_

# cc_lasso = np.array(cc_lasso)

In [ ]:
#Plot coefficient amplitudes versus regularization parameter.
fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(111)

for i in range(len(cc_lasso[0])):
    ax.semilogx(lls, cc_lasso[:, i], label='$\omega_{}$'.format(i), lw=3, alpha=0.6)
ax.legend(ncol=3, fontsize=16)
ax.set_xlabel('$\lambda$')
ax.set_ylabel('Parameter value')

In [ ]:
# Plot all coefficients

# Instanciate Figure and Axes
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)

# Create meshgrid based on values of lambda
Xp, Yp = np.meshgrid(np.arange(10)-0.5, lls)
Zp = np.where(np.abs(cc_lasso)>0, np.log(np.abs(cc_lasso)), -np.inf)

# Vertical lines
for i in range(10):
    ax.axvline(i+0.5, color='0.8', lw=0.5)
    
# Color plot
pcol = ax.pcolor(Xp, Yp, Zp)


# Set log scale
ax.set_yscale('log')

# Colorbar and labels
plt.colorbar(pcol, label='$\log\left(|\omega_j|\\right)$')
ax.set_xlabel('$j$', fontsize=18)
ax.set_ylabel('$\lambda$', fontsize=18)

ax.set_title('Lasso shrinkage plot')

In [ ]:
j = [0, 3, 5]
plt.figure(figsize=(9, 4))

plt.semilogx(lls, cc_lasso[:, j])
    
plt.axhline(0, color='r', ls=':')
plt.xlabel('Regularization parameter, $\lambda$')

if len(j) > 1:
    plt.legend(plt.gca().lines, ['$\omega_{}$'.format(jj) for jj in j], fontsize=16)
    plt.ylabel('Parameter value')
else:
    plt.ylabel('$\omega_{}$'.format(j))

plt.show()

Como se puede ver, a medida que aumentamos el término de regularización, algunos parámetros van estrictamente a cero. De esta manera, la regresión Lasso también funciona como una especie de herramienta de selección de modelos.

## Interpretación geométrica

$$
E_\text{ridge}(\boldsymbol{\omega}; \lambda) = \frac{1}{2} \sum_{i=1}^{N} \left\{y(x_i, \boldsymbol{\omega}) - t_i\right\}^2
 + \frac{\lambda}{2} \sum_{i=1}^M \omega_i^2\;\;, 
$$

Para entender el comportamiento diferente entre Ridge y Lasso, usemos un lindo plot del [libro de Bishop](https://www.microsoft.com/en-us/research/people/cmbishop/prml-book/).

<table>
    <tr>
        <td>
      <img src="images/05_Regularization/ridge.png" width=250px>
        </td>
        <td>
            <img src="images/05_Regularization/lasso.png" width=250px>
        </td>
    </tr>
    </table>

Donde el punto azul en el centro de los círculos representa la solución no regularizada, es decir, la solución OLS, y $\omega^*$ es el vector de parámetros óptimo bajo penalización.

***
<font size=4>Ahora a ustedes! Preparen sus teclados!</font>
<!-- ### Prepare your keyboards! -->
***

* Utilicen validación cruzada (`GridSearchCV` o `RandomizedSearchCV`) para encontrar los valores óptimos de los hiperparámetros (orden polinómico M y parámetro de regularización $\lambda = 2 \alpha$.
* Hagan esto para **Ridge** y **Lasso** de forma independiente.
* Hacer un gráfico de la dependencia del error de testeo con los valore de los hiperparámetros. Dado que solo hay dos hiperparámetros, tal vez quieras probar `plt.pcolor` o `plt.imshow`.
* Registrar los valores de los mejores hiper-parámetros.

**Nota**: recuerdá que podés guardar las imágenes que quieras con la función `save_fig` definida arriba.
***

# Bias-Variance tradeoff


In [ ]:
N_SAMPLES = 15
x = np.linspace(0,1,num=N_SAMPLES).reshape(-1,1)

def ground_truth(x):
    return 4 * x+ np.sin(x*6)

N_REPEAT = 50

all_data = np.empty([N_REPEAT, N_SAMPLES])

# Repeat realization of data many times
for i in range(N_REPEAT):
    t =  ground_truth(x.flatten()) + 0.5*np.random.randn(N_SAMPLES)
#     x_train, x_test, t_train, t_test = train_test_split(x, t)
    
    all_data[i] = t

In [ ]:
from sklearn.metrics import mean_squared_error

lls = [0.0, 1e-3, 10.0]

all_predictions = np.empty([len(lls), N_REPEAT, len(x_)])
for i, data in enumerate(all_data):
    for j, ll in enumerate(lls):
        reg = ridge(9, ll)
        reg.fit(x, data)
        all_predictions[j, i] = reg.predict(x_)

In [ ]:
print(all_predictions.shape)

In [ ]:
n_columns = 3
N_ROWS = 2

fig = plt.figure(figsize=(16, 4 * N_ROWS))
axs = fig.subplots(ncols=n_columns, nrows=N_ROWS, gridspec_kw={'wspace': 0.3})

# randomly choose simulations to draw
ind = np.arange(all_predictions.shape[1])
np.random.shuffle(ind)

for i, ax in enumerate(axs.flatten()):
    # Plot ground truth
    ax.plot(x_, ground_truth(x_), color='k', lw=3, label='Ground truth')
    
    # Plot data
    ax.plot(x, all_data[ind[i]], 'ob', mfc=None, ms=10)
    
    ax.legend(loc=0)
    
    for j in range(len(lls)):
        ax.plot(x_, all_predictions[j, ind[i]], alpha=0.6, 
                label='$\lambda = {:.1e}$'.format(lls[j]))
    
    
    #Label
    ax.set_xlabel('X')
    ax.set_ylabel('t')

In [ ]:
N_PLOT = 30

fig = plt.figure(figsize=(16, 8))
axs = fig.subplots(nrows=1, ncols=3, gridspec_kw={'wspace': 0.3})

# randomly choose simulations to draw
ind = np.arange(all_predictions.shape[1])
np.random.shuffle(ind)

for i, ax in enumerate(axs):
    ax.plot(x_, ground_truth(x_), color='k', lw=3, label='Ground truth')
    ax.plot(x_, all_predictions[i, ind[:N_PLOT]].T, color='Red', alpha=0.15)
    ax.legend(loc=0)
    
    #Label
    ax.set_xlabel('X')
    ax.set_ylabel('t')
    
    # Title
    axs[i].set_title('Reg. Param: {:.1e}'.format(lls[i]))

## Cálculo completo de sesgo y varianza

In [ ]:
# Repeat for finer grid of lambdas
lls = np.logspace(-5, np.log10(5), 40)
lls = np.concatenate([np.array([0,]), lls])

all_predictions_full = np.empty([len(lls), N_REPEAT, len(x_)])
for i, data in enumerate(all_data):
    for j, ll in enumerate(lls):
        reg = ridge(9, ll)
        reg.fit(x, data)
        all_predictions_full[j, i] = reg.predict(x_)
        
# All predictions has shape n_lambdas x n_repeats x len(x_)

In [ ]:
all_predictions_full.shape

In [ ]:
# Noise added to measurements
irreduc_err = 0.1

# Compute mean prediction over data realisations
Ed_y = np.average(all_predictions_full, axis=1)

# Compute bias (difference beween mean prediction and truth)
bias2 = (Ed_y - ground_truth(x_.flatten()))**2

# Variance
var = np.mean((Ed_y[:, np.newaxis, :] - all_predictions_full)**2, axis=1)

# Total error
err = bias2 + var + irreduc_err**2

In [ ]:
print(var.shape, bias2.shape, err.shape, lls.shape)

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111)
# Plot in same plot, averaged over X
ax.semilogx(lls, var.mean(axis=1), color='r', label='variance', lw=4)
ax.semilogx(lls, bias2.mean(axis=1), color='b', label='bias$^2$', lw=4)
ax.semilogx(lls, err.mean(axis=1), label='Expected error', lw=4, color='k')

# Title and labels
ax.legend(fontsize=16)
ax.set_xlabel('Regularization parameter', size=16)
ax.set_ylabel('Error', size=16)

# ax.invert_xaxis()